#### Client의 행동을 따라가보자
- Client가 어떻게 행동하는지 데이터 살펴보기
- Client Behavior 데이터 만들기 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [2]:
#데이터가져오기
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#구매데이터의 client 수 != 고객데이터의 client 수 
#구매데이터의 client 수 != 검색데이터의 client 수
print('Product shape: ',Product.shape)
print('Custom shape: ',Custom.shape)
print('Search1 shape: ',Search1.shape)
print('Session shape: ',Session.shape)
print()
print('구매에 참여한 client 수: ',len(Product['CLNT_ID'].value_counts()))
print('고객데이터에 있는 unique한 client 수: ',len(Custom['CLNT_ID'].value_counts()))
print('세션에 참여한 unique한 client 수: ',len(Session['CLNT_ID'].value_counts()))
print('검색에 참여한 client 수: ', len(Search1['CLNT_ID'].value_counts()))

Product shape:  (5024906, 8)
Custom shape:  (671679, 3)
Search1 shape:  (2884943, 4)
Session shape:  (2712907, 9)

구매에 참여한 client 수:  922737
고객데이터에 있는 unique한 client 수:  671679
세션에 참여한 unique한 client 수:  922737
검색에 참여한 client 수:  511477


- 세션에 참여한 client 수와 구매에 참여한 client 수는 동일하다 
- 상품을 구매했을지라도 검색을 하지 않은 client들이 있다 ) 922737 - 511477 만큼
- 고객데이터에 모든 client들의 정보가 있는 것은 아니다 

In [4]:
temp1 = pd.merge(Search1, Product, how = 'right' , on = 'CLNT_ID')

In [5]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [6]:
temp2 = pd.merge(Custom, Product, how = 'right' , on = 'CLNT_ID')

In [7]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [8]:
#null값 체크 --> 이건 unique한 건 아니지만 그래도 null값이 있다는 걸 의미함 
print(temp2['CLNT_AGE'].isnull().sum().sum())

1036319


- Client ID, 날짜(세션일자), 세션 ID, HIT번호, 검색키워드, 검색건수, 상품명, 구매건수 

- CLNT_ID(Product), SESS_DT(Session), SESS_ID(세션ID), HITS_SEQ(Product)
- KWD_NM(Search1), SEARCH_CNT(Search1), PD_NM(Master), PD_BUY_CT(Product)

#### Search1의 client < custom의 client < Product의 client = Session의 Client  (부분집합 관계)
#### 검색을 한 client만을 기준으로 table 작성 

- 데이터 전처리 : outlier 제거
- HIT_SEQ가 1인 것 제거하기 

In [27]:
#HIT_SEQ가 1인 Column 제거하기 (outlier로 간주하라고 하였음)

idx = (Product['HITS_SEQ'] != 1)
Product = Product[idx]
print(len(Product['CLNT_ID'].value_counts())) 

922618


In [28]:
Product[Product['CLNT_ID'] == 4758889]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
533815,4758889,10607258,24,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
597057,4758889,10607258,195,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
655687,4758889,10607258,298,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
661037,4758889,10730334,76,692624,색상:블랙 / 1개,LG전자,"45,340",1
4954520,4758889,10607258,298,776358,사이즈:28 / 1개,게스(진),"39,000",1
5022807,4758889,10607258,195,776358,사이즈:28 / 1개,게스(진),"39,000",1


- Search1의 Client id를 기준으로 merge

In [39]:
#Product에서 outlier를 제거해 줬으므로 Product의 client에 맞춘다
data1 = pd.merge(Product, Session)
print(len(data1['CLNT_ID'].value_counts()))
print('아주 조금 줄었음')

922618
아주 조금 줄었음


In [30]:
#상품정보를 추가해준다
data2 = pd.merge(data1, Master, how = 'left', on = 'PD_C')
print(len(data1['CLNT_ID'].value_counts()))

922618


In [17]:
#세션과 구매에 참여한 client를 기준으로 data2과 Search1을 left outer join 해 준다 
#세션과 구매에 참여한 client일지라도 검색을 하지 않은 client가 존재한다
#data2 = data2[['CLNT_ID','SESS_ID','HITS_SEQ','SESS_DT','TOT_PAG_VIEW_CT','PD_NM','CLAC1_NM','PD_BUY_CT','ZON_NM']]

In [40]:
idx = data2['CLNT_ID'] == 4758889
data2[idx]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
966746,4758889,10607258,24,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,하이넥 레이스 플레어 원피스 (TZ-OQT0064) - 베이지 / F,여성의류,여성의류전신,여성원피스
966747,4758889,10607258,195,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,하이넥 레이스 플레어 원피스 (TZ-OQT0064) - 베이지 / F,여성의류,여성의류전신,여성원피스
966748,4758889,10607258,298,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,하이넥 레이스 플레어 원피스 (TZ-OQT0064) - 베이지 / F,여성의류,여성의류전신,여성원피스
966749,4758889,10607258,298,776358,사이즈:28 / 1개,게스(진),"39,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,여성 원워시 슈퍼스키니(YG1D9043) - 26,남성의류,남성의류하의,남성캐주얼바지
966750,4758889,10607258,195,776358,사이즈:28 / 1개,게스(진),"39,000",1,82,20180406,151.0,"2,513",mobile,Gyeonggi-do,Paju-si,여성 원워시 슈퍼스키니(YG1D9043) - 26,남성의류,남성의류하의,남성캐주얼바지
1176448,4758889,10730334,76,692624,색상:블랙 / 1개,LG전자,"45,340",1,75,20180404,41.0,"1,134",mobile,Gyeonggi-do,Paju-si,[공식인증점]HBS-510 톤플러스(TONE PLUS) 블루투스 이어폰 /초경량디자...,모바일,모바일액세서리,이어폰/헤드폰


In [35]:
#Search1의 client기준으로 병합
#client -> 검색 -> 구매
Client_Behavior = pd.merge(data2, Search1, on=['CLNT_ID', 'SESS_ID']) 

In [37]:
print(len(Client_Behavior['CLNT_ID'].value_counts()))

511447


In [33]:
Client_Behavior.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT
0,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,입생로랑,3
1,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,이솝,3
2,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다 스무드,1
3,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다,23
4,4142395,6158159,23,558739,상품명:이솝 제라늄 리프 바디 클렌저,[이솝],"56,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,이솝 제라늄 리프 바디 클렌저 500ml - 이솝 제라늄 리프 바디 클렌저,퍼스널케어,바디케어,바디워시,입생로랑,3


In [41]:
Client_Behavior[Client_Behavior['CLNT_ID']== 4142395]  

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT
0,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,입생로랑,3
1,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,이솝,3
2,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다 스무드,1
3,4142395,6158159,85,179538,(not set),[아베다],"39,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다,23
4,4142395,6158159,23,558739,상품명:이솝 제라늄 리프 바디 클렌저,[이솝],"56,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,이솝 제라늄 리프 바디 클렌저 500ml - 이솝 제라늄 리프 바디 클렌저,퍼스널케어,바디케어,바디워시,입생로랑,3
5,4142395,6158159,23,558739,상품명:이솝 제라늄 리프 바디 클렌저,[이솝],"56,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,이솝 제라늄 리프 바디 클렌저 500ml - 이솝 제라늄 리프 바디 클렌저,퍼스널케어,바디케어,바디워시,이솝,3
6,4142395,6158159,23,558739,상품명:이솝 제라늄 리프 바디 클렌저,[이솝],"56,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,이솝 제라늄 리프 바디 클렌저 500ml - 이솝 제라늄 리프 바디 클렌저,퍼스널케어,바디케어,바디워시,아베다 스무드,1
7,4142395,6158159,23,558739,상품명:이솝 제라늄 리프 바디 클렌저,[이솝],"56,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,이솝 제라늄 리프 바디 클렌저 500ml - 이솝 제라늄 리프 바디 클렌저,퍼스널케어,바디케어,바디워시,아베다,23
8,4142395,6158159,43,176911,(not set),[아베다],"90,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[6월] 스무드 인퓨전 샴푸 리터 세트,퍼스널케어,헤어케어,린스/컨디셔너,입생로랑,3
9,4142395,6158159,43,176911,(not set),[아베다],"90,000",1,1,20180619,...,"6,524",desktop,Seoul,Seoul,[6월] 스무드 인퓨전 샴푸 리터 세트,퍼스널케어,헤어케어,린스/컨디셔너,이솝,3


In [42]:
#Client_Behavior 데이터 내보내기
Client_Behavior.to_csv("C:/Users/lsh93/Desktop/data/Client_Behavior.csv", encoding="UTF-8")

- 상품 구매 count

In [44]:
Client_Behavior[]

KeyError: 'CLAC1_NM'